In [1]:
import requests
import pandas as pd

from utils import solve_dir, solve_path

from config import KEY_API_SOF

In [2]:
GENERATED_DATA_DIR = solve_dir('generated_data')
GENERATED_DATA_DIR = solve_path('fontes_recursos', GENERATED_DATA_DIR)
GENERATED_DATA_DIR = solve_dir(GENERATED_DATA_DIR)

In [3]:
auth_headers = {'Authorization' :  f'Bearer {KEY_API_SOF}'}

In [4]:
def build_url(endpoint, **kwargs):
    
    domain = (
        'https://gatewayapi.prodam.sp.gov.br:443/'
        'financas/orcamento/sof/v3.0.1/'
        )
    
    if not kwargs:
        return domain + endpoint
    else:
        query = '&'.join([f'{key}={val}' for key, val in kwargs.items()])
        return domain + endpoint + '?' + query

In [5]:
def get_request(url, headers=auth_headers):
    
    with requests.get(url, headers=headers) as r:
        return r.json()

In [6]:
url = build_url('fonteRecursos', anoExercicio=2022)

In [7]:
resp = get_request(url)

In [8]:
assert resp['metadados']['txtStatus']=='OK'

In [9]:
data= resp['lstFontesRecursos']

In [10]:
df_fontes = pd.DataFrame(data)

In [11]:
def save_df(df, fname, folder):
    
    fname = solve_path(fname, folder)
    
    try:
        df.to_csv(fname, sep =';', decimal=',', 
                  encoding='latin-1', index=False)
    except UnicodeEncodeError:
        df.to_csv(fname, sep =';', decimal=',', 
                  encoding='utf-8', index=False)

In [12]:
save_df(df_fontes, 'fontes_recursos_sof.csv', folder = GENERATED_DATA_DIR)